In [1]:
import findspark
findspark.init()
findspark.find()

from IPython.display import *
display(HTML("<style>pre {white-space: pre !important; }</style>"))

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
            SparkSession
                .builder
                .appName("DeltaLakeApp")
    
                .master("local[4]")    
                .config("spark.dynamicAllocation.enabled", "false")     
                .config("spark.sql.adaptive.enabled", "false")
                .config("sparl.sql.optimizer.dynamicPartitionPruning", "false")
                .getOrCreate()
        )

sc = spark.sparkContext

spark

In [3]:
# Create schema for Yellow Taxi Data
 
taxiSchema = (
                    StructType
                    ([ 
                        StructField("VendorId"               , IntegerType()   , True),
                        StructField("lpep_pickup_datetime"   , TimestampType() , True),
                        StructField("lpep_dropoff_datetime"  , TimestampType() , True),                            
                        StructField("passenger_count"        , DoubleType()    , True),
                        StructField("trip_distance"          , DoubleType()    , True),
                        StructField("RatecodeID"             , DoubleType()    , True),                            
                        StructField("store_and_fwd_flag"     , StringType()    , True),
                        StructField("PULocationID"           , IntegerType()   , True),
                        StructField("DOLocationID"           , IntegerType()   , True),                            
                        StructField("payment_type"           , IntegerType()   , True),                            
                        StructField("fare_amount"            , DoubleType()    , True),
                        StructField("extra"                  , DoubleType()    , True),
                        StructField("mta_tax"                , DoubleType()    , True),
                        StructField("tip_amount"             , DoubleType()    , True),
                        StructField("tolls_amount"           , DoubleType()    , True),
                        StructField("improvement_surcharge"  , DoubleType()    , True),
                        StructField("total_amount"           , DoubleType()    , True),
                        StructField("congestion_surcharge"   , DoubleType()    , True),
                        StructField("airport_fee"            , DoubleType()    , True)
                    ])
               )

In [4]:
yellowTaxiDf = (
    spark
    .read
    .option("header", "true")
    .schema(taxiSchema)
    .csv("C:\DataFiles\YellowTaxis_202210.csv")
)

yellowTaxiDf.printSchema()

root
 |-- VendorId: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [10]:
(yellowTaxiDf
 .write
 .partitionBy("PULocationID")
 .option("header", "true")
 .option("dateFormat", "yyyy-MM-dd HH:mm:ss.s")
 .mode("overwrite")
 .format("csv")
 .option("path", "C:\DataFiles\PruningOutput2\YellowTaxisOutput.csv")
 .saveAsTable("YellowTaxisZone")
)

In [11]:
taxiZoneSchema = "LocationID INT, Borough STRING, Zone STRING, serviceZone STRING"

taxiZonesDf = (
    spark
    .read
    .schema(taxiZoneSchema)
    .csv("C:\DataFiles\TaxiZones.csv")
)

(taxiZonesDf
 .write
 .option("header", "true")
 .option("dateFormat", "yyyy-MM-dd HH:mm:ss.s")
 .mode("overwrite")
 .format("csv")
 .option("path", "C:\DataFiles\PruningOutput2\TaxiZones.csv")
 .saveAsTable("TaxisZones")
)

In [13]:
spark.sql("""
select * from YellowTaxisZone yt join TaxisZones tz on yt.PULocationID = tz.LocationID where yt.PULocationID = 1
""").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------+----------+-------+--------------+-----------+
|VendorId|lpep_pickup_datetime|lpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|PULocationID|LocationID|Borough|          Zone|serviceZone|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------+----------+-------+--------------+-----------+
|       2| 2022-10-09 18:42:58|  2022-10-09 18:43

In [16]:
spark.sql("""
select * from Taxiszones where Borough = 'EWR'
""").show()

+----------+-------+--------------+-----------+
|LocationID|Borough|          Zone|serviceZone|
+----------+-------+--------------+-----------+
|         1|    EWR|Newark Airport|        EWR|
+----------+-------+--------------+-----------+



In [20]:
spark.sql("""
select * from YellowTaxisZone yt join TaxisZones tz on yt.PULocationID = tz.LocationID where tz.Borough = 'EWR'
""").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------+----------+-------+--------------+-----------+
|VendorId|lpep_pickup_datetime|lpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|PULocationID|LocationID|Borough|          Zone|serviceZone|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------+----------+-------+--------------+-----------+
|       2| 2022-10-09 18:42:58|  2022-10-09 18:43